<a href="https://colab.research.google.com/github/Rhymster16/hello/blob/main/backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Resume Skills Extractor in Google Colab (Fixed Version)
!pip install PyPDF2 python-docx
from IPython.display import display, HTML
import ipywidgets as widgets
import re
import PyPDF2 # This import requires PyPDF2 to be installed
import io
from base64 import b64decode
from docx import Document # Explicitly import Document for docx handling

# Install required packages (only needed once)
# Ensure this cell is run and completes successfully before proceeding


# Skills database (expanded)
SKILLS_DB = [
    'python', 'javascript', 'java', 'c++', 'html', 'css',
    'react', 'angular', 'vue', 'sql', 'mysql', 'mongodb',
    'docker', 'kubernetes', 'aws', 'azure', 'gcp',
    'flask', 'django', 'node.js', 'express',
    'machine learning', 'data analysis', 'pandas', 'numpy',
    'git', 'github', 'jenkins', 'ci/cd'
]

def extract_text(file_content, filename):
    try:
        if filename.lower().endswith('.pdf'):
            # Using PdfReader for PyPDF2 v3+
            pdf = PyPDF2.PdfReader(io.BytesIO(file_content))
            # Join pages, ensuring text exists
            return " ".join(page.extract_text() for page in pdf.pages if page.extract_text())
        elif filename.lower().endswith(('.txt', '.text')):
            return file_content.decode('utf-8')
        elif filename.lower().endswith('.docx'):
            # Ensure python-docx is imported here if not at the top
            # from docx import Document # Already imported at the top
            doc = Document(io.BytesIO(file_content))
            return "\n".join(paragraph.text for paragraph in doc.paragraphs)
        else:
            raise ValueError("Unsupported file format")
    except Exception as e:
        # Catch PyPDF2 specific errors or docx errors and re-raise with more context
        raise ValueError(f"Error reading file {filename}: {str(e)}")


def extract_skills(text):
    text = text.lower()
    found_skills = set()
    for skill in SKILLS_DB:
        # Use word boundaries to avoid matching substrings (e.g., 'java' in 'javascript')
        # re.escape is important if skills contain special regex characters like '.' or '+'
        if re.search(rf'\b{re.escape(skill)}\b', text):
            found_skills.add(skill)
    return sorted(found_skills)

def on_upload(change):
    # Check if a file was actually uploaded
    # ipywidgets.FileUpload.value is a dict like object {filename: {'metadata': {}, 'content': bytes}}
    # or {} if no file is selected or cleared.
    if not upload_button.value:
        output_area.value = "<p style='color:orange;'>Please upload a file.</p>"
        return

    try:
        # Access the uploaded file details
        # .value in ipywidgets.FileUpload gives a dictionary-like object
        # We expect only one file due to multiple=False
        file_key = list(upload_button.value.keys())[0] # Get the filename key
        file_data = upload_button.value[file_key]

        file_content = file_data['content']
        filename = file_key # The key is the filename

        # Handle potential base64 encoding in some environments (like Colab)
        # This check might be redundant with recent ipywidgets versions, but safe to keep
        if isinstance(file_content, dict) and 'content' in file_content:
             file_content = b64decode(file_content['content'])
        # Ensure file_content is bytes for io.BytesIO
        elif not isinstance(file_content, bytes):
             # Assuming it might be a memoryview or other bytes-like object, convert to bytes
             # This might be the case in some environments
             file_content = bytes(file_content)


        text = extract_text(file_content, filename)
        skills = extract_skills(text)

        skills_html = "".join(
            f'<span style="background:#e0e0e0; padding:5px 10px; margin:5px; '
            f'border-radius:3px; display:inline-block;">{skill}</span>'
            for skill in skills) or "<p>No skills detected</p>"

        # Display results
        output_area.value = f"""
        <div style="margin-top:20px; padding:15px; border:1px solid #ddd; border-radius:5px; background-color:#f9f9f9;">
            <h3 style="margin-top:0; color:#333;">Extracted Skills</h3>
            <div>{skills_html}</div>
            <p style="margin-top:15px;"><strong>Processed file:</strong> {filename}</p>
            <p><strong>Characters processed:</strong> {len(text):,}</p>
        </div>
        """
    except Exception as e:
        # Display error message
        output_area.value = f"""
        <div style="color:red; padding:10px; border:1px solid red; border-radius:5px; background-color:#ffe0e0;">
            <strong>Error processing file:</strong> {str(e)}
            <p>Please try a different file or format (PDF, DOCX, or TXT).</p>
        </div>
        """
    finally:
        # Clear the file upload value after processing to allow re-uploading the same file
        # The 'value' trait is read-only, so we use the clear() method
        upload_button.clear()


# UI Components
upload_button = widgets.FileUpload(
    description="📁 Upload Resume",
    accept='.pdf,.txt,.docx',
    multiple=False,
    style={'button_color': '#4CAF50', 'description_width': 'initial'}
)

output_area = widgets.HTML()

# Link the upload button to the on_upload function
upload_button.observe(on_upload, names='value')

# Display the UI
display(HTML("""
<div style="font-family:Arial; max-width:800px; margin:auto; padding:20px; border-radius:8px; background-color:#ffffff; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
    <h1 style="color:#2c3e50; text-align:center;">Resume Skills Extractor</h1>
    <p style="text-align:center; color:#555;">Upload your resume (PDF, DOCX, or TXT) to extract skills:</p>
</div>
"""))
display(upload_button)
display(output_area)

FileUpload(value={}, accept='.pdf,.txt,.docx', description='📁 Upload Resume', style=ButtonStyle(button_color='…

HTML(value='')

AttributeError: 'FileUpload' object has no attribute 'clear'

In [6]:
!pip install -q colab-git
from google.colab import drive
drive.mount('/content/drive')

ERROR: Could not find a version that satisfies the requirement colab-git (from versions: none)
ERROR: No matching distribution found for colab-git


MessageError: Error: credential propagation was unsuccessful